In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import mne
import numpy as np
from copy import deepcopy
import seaborn as sns
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info

plot_params = dict(surface='white', hemi='both', verbose=0)

# Forward Model

In [2]:
info = get_info()
info['sfreq'] = 100
fwd = create_forward_model(info=info)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.0s remaining:    1.7s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.1s remaining:    0.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished


# Simulate

In [6]:
n_samples = 10000
settings = dict(duration_of_trial=1.0)

sim_lstm = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=n_samples)

Simulate Source


  0%|          | 0/1000 [00:00<?, ?it/s]

Converting Source Data to mne.SourceEstimate object


  0%|          | 0/1000 [00:00<?, ?it/s]


Project sources to EEG...
	took 0.79 seconds

Create EEG trials with noise...


  0%|          | 0/1000 [00:00<?, ?it/s]


Convert EEG matrices to a single instance of mne.Epochs...


## Swap true sources with eloreta predictions

In [20]:
from esinet.util import wrap_mne_inverse
pred_elor = wrap_mne_inverse(fwd, sim_lstm, method='eLORETA')

# Scale
for i, src in enumerate(pred_elor):
    pred_elor[i].data = src.data / np.abs(src.data).max(axis=0)

sim_lstm.source_data = pred_elor

# Build & train LSTM network

In [21]:
epochs = 150
patience = 10
activation_funcion = 'relu'
loss = 'huber'
dropout = 0.2

# Dense net
model_params = dict(activation_function=activation_funcion, n_dense_layers=2, 
    n_dense_units=400, n_lstm_layers=0, model_type='v2')
train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
    dropout=dropout, loss=loss, optimizer='adam', return_history=True,
    batch_size=8)
net_dense = Net(fwd, **model_params)
_, history_dense = net_dense.fit(sim_lstm, **train_params)

# LSTM v2
model_params = dict(activation_function=activation_funcion, n_lstm_layers=1, 
    n_lstm_units=200, n_dense_layers=0, 
    model_type='v2')
train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
    dropout=0, loss=loss, return_history=True, learning_rate=0.001,
    batch_size=8)

net_lstm = Net(fwd, **model_params)
_, history_lstm = net_lstm.fit(sim_lstm, **train_params)


models = [net_dense, net_lstm]
model_names = ['Dense', 'LSTM']

Model: "LSTM_v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FC_0 (TimeDistributed)       (None, None, 400)         24800     
_________________________________________________________________
Drop_0 (Dropout)             (None, None, 400)         0         
_________________________________________________________________
FC_1 (TimeDistributed)       (None, None, 400)         160400    
_________________________________________________________________
Drop_1 (Dropout)             (None, None, 400)         0         
_________________________________________________________________
FC_Out (TimeDistributed)     (None, None, 1284)        514884    
Total params: 700,084
Trainable params: 700,084
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
113/113 [==============================] - 5s 42ms/step - loss: 0.2381 - mae: 0.5486 - val_loss: 0.0687 - v

# Plot single ground truth and predictions

In [35]:
epp = mne.EpochsArray(np.random.randn(1,61,100), info, verbose=0)
from time import time
start = time()
for _ in range(100):
    net_lstm.predict(epp)
end = time()
print((end-start)/100)

# sim_lstm.eeg_data.get_data().shape

0.20769521474838257


In [22]:
from esinet.util import wrap_mne_inverse
import seaborn as sns
%matplotlib qt
sns.reset_orig()
plot_params = dict(surface='white', hemi='both', verbose=0, 
    clim=dict(kind='percent', pos_lims=[20, 30, 100]))

# settings_eval = dict(duration_of_trial=1, target_snr=(0.5, 10), extents=(15, 35), number_of_sources=(1, 20))
settings_eval = dict(duration_of_trial=1, target_snr=9999, extents=(15, 35), number_of_sources=2)

# Simulate new data
sim_test = Simulation(fwd, info, settings=settings_eval).simulate(1)
idx = 0
# Predict sources using the esinet models
predictions = [model.predict(sim_test) for model in models]
# Predict sources with classical methods
prediction_elor_data = wrap_mne_inverse(fwd, sim_test)[idx].data.astype(np.float32)
prediction_elor = deepcopy(predictions[0])
prediction_elor.data = prediction_elor_data / np.abs(np.max(prediction_elor_data))
prediction_mne_data = wrap_mne_inverse(fwd, sim_test, method='MNE')[idx].data.astype(np.float32)
prediction_mne = deepcopy(predictions[0])
prediction_mne.data = prediction_mne_data / np.abs(np.max(prediction_mne_data))
# Get predictions and names in order
predictions.append(prediction_elor)
predictions.append(prediction_mne)
model_names.append('eLORETA')
model_names.append('MNE')

# Plot True Source
brain = sim_test.source_data[idx].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title')
# Plot True EEG
evoked = sim_test.eeg_data[idx].average()
evoked.plot()
evoked.plot_topomap(title='Ground Truth')
evoked = util.get_eeg_from_source(sim_test.source_data[idx], fwd, info, tmin=0.)
evoked.plot_topomap(title='Ground Truth Noiseless')

# Plot predicted sources
for model_name, prediction in zip(model_names, predictions):
    brain = prediction.plot(**plot_params)
    brain.add_text(0.1, 0.9, model_name, 'title')
    # Plot predicted EEG
    # evoked_esi = util.get_eeg_from_source(prediction, fwd, info, tmin=0.)
    # evoked_esi.plot()
    # evoked_esi.plot_topomap(title=model_name)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

	took 0.16 seconds


  0%|          | 0/1 [00:00<?, ?it/s]

  File "C:\Users\lukas\virtualenvs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [0.38828332 0.43662978 0.66681116]


  File "C:\Users\lukas\virtualenvs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [1.82880090e-11 2.08840266e-11 2.97204482e-11]
Using control points [0.38828332 0.43662978 0.66681116]
Using control points [1.82880090e-11 2.08840266e-11 2.97204482e-11]


  File "C:\Users\lukas\virtualenvs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [9.85546530e-12 3.93145593e-11 9.82169077e-10]


# Calculate Evaluation Metrics

In [39]:
from esinet.evaluate import eval_mean_localization_error, eval_nmse, eval_auc, eval_mse
from esinet.util import wrap_mne_inverse
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm

# Simulate
settings = dict(duration_of_trial=0.2, target_snr=(0.5, 10), number_of_sources=1)

sim_lstm_test = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=5000)

# Predict
# models = [net_lstm, net_lstm_big]
print('predict esinets...')
# model_names = [model.model.name for model in models]
predictions = [model.predict(sim_lstm_test) for model in models]

print('predict elor & MNE')
pred_elor = wrap_mne_inverse(fwd, sim_lstm_test, method='eLORETA')
# model_names.append('eLORETA')
predictions.append(pred_elor)

pred_mne = wrap_mne_inverse(fwd, sim_lstm_test, method='MNE')
# model_names.append('MNE')
predictions.append(pred_mne)

size = 500
pos = util.unpack_fwd(fwd)[2]
distance_matrix = cdist(pos, pos)

mean_localization_errors = []
aucs = []
nmses = []
mses = []
true_sources = np.stack([src.data for src in sim_lstm_test.source_data], axis=0)
true_sources = util.collapse(true_sources)
choice = np.random.choice(np.arange(true_sources.shape[0]), size=size, replace=False)
true_sources = true_sources[choice]
# true_sources = true_sources[:size]

for prediction in predictions:
    predicted_sources = util.collapse(np.stack([src.data for src in prediction], axis=0))
    
    predicted_sources = predicted_sources[choice]
    # predicted_sources = predicted_sources[:size]
    
    print('mle calculation....')
    mean_localization_error = [eval_mean_localization_error(true_source, predicted_source, pos, distance_matrix=distance_matrix) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    auc = [eval_auc(true_source, predicted_source, pos, epsilon=0.05,n_redraw=25) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    nmse = [eval_nmse(true_source, predicted_source) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    mse = [eval_mse(true_source, predicted_source) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    
    mean_localization_errors.append(mean_localization_error)
    aucs.append(auc)
    nmses.append(nmse)
    mses.append(mse)

aucs_far = [auc[:, 1] for auc in np.array(aucs)]
aucs_close = [auc[:, 0] for auc in np.array(aucs)]

Simulate Source


  0%|          | 0/5000 [00:00<?, ?it/s]

Converting Source Data to mne.SourceEstimate object


  0%|          | 0/5000 [00:00<?, ?it/s]


Project sources to EEG...

Create EEG trials with noise...


...

mle calculation....


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

# Plot Evaluation Metrics

In [40]:
import seaborn as sns; sns.set(style='whitegrid')
%matplotlib qt
xticks = dict(ticks=np.arange(len(model_names)), labels=model_names)
plot = sns.boxplot  # violinplot
variables = [nmses, mses, aucs_far, aucs_close, mean_localization_errors]
names = ['Normalized Mean Squared Errors', 'Mean Squared Errors', 'Far area under the curve', 'Close area under the curve', 'Mean Localization Errors']
plt.figure(figsize=(12, 10))
    
subplot_nums = np.arange(321, 326)
for variable, name, num in zip(variables, names, subplot_nums):
    plt.subplot(num)
    plot(data=variable)
    plt.title(name)
    plt.xticks(**xticks)
plt.tight_layout()


In [7]:
from scipy.stats import pearsonr
stretched_indices = np.repeat(np.arange(sim_lstm_test.n_samples), 20)
param = "target_snr"  # "number_of_sources"
names = ['Normalized Mean Squared Errors', 'Mean Squared Errors', 'Far area under the curve', 'Close area under the curve', 'Mean Localization Errors']
variables = [nmses, mses, aucs_far, aucs_close, mean_localization_errors]
subplot_nums = np.arange(321, 326)
model_names = ['Dense', 'LSTM', 'eLORETA', 'MNE']
n_sources = sim_lstm_test.simulation_info.iloc[stretched_indices[choice]][param].values

plt.figure()
for variable, name, num in zip(variables, names, subplot_nums):
    plt.subplot(num)
    for i, model_name in enumerate(model_names[:4]):
        try:
            plt.scatter(n_sources, np.array(variable)[i, :], label=model_name + f' m={np.array(variable)[i,:].mean():.2f}, r={pearsonr(n_sources, np.array(variable)[i, :])[0]:.2f}')
        except:
            plt.scatter(n_sources, np.array(variable)[i, :], label=model_name + f' m={np.nanmean(np.array(variable)[i,:]):.2f}')
            
    plt.xlabel(param)
    plt.ylabel(name)
    plt.legend()


In [102]:
import seaborn as sns
%matplotlib qt
model_names[0] = 'Dense'

data = {model_name: nmse for model_name, nmse in zip(model_names, nmses)}
plt.figure()
sns.kdeplot(data=data, multiple='stack')
plt.title('Normalized Mean Squared Errors')

plt.figure()
data = {model_name: mse for model_name, mse in zip(model_names, mses)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Mean Squared Errors')


plt.figure()
data = {model_name: auc for model_name, auc in zip(model_names, aucs_far)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Far area under the curve')


plt.figure()
data = {model_name: auc for model_name, auc in zip(model_names, aucs_close)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Close area under the curve')

plt.figure()
data = {model_name: mle for model_name, mle in zip(model_names, mean_localization_errors)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Close area under the curve')
plt.title('Mean Localization Errors')

plt.tight_layout()

# Performance per Difficulty Evaluation

In [ ]:
metrics = [nmses, mean_localization_errors, aucs_far, aucs_close]
metric_names = ['nmses', 'mean_localization_errors', 'aucs_far', 'aucs_close']

covariates = ['target_snr', 'number_of_sources']

n_samples = int(np.array(nmses).shape[1] / 20)
for covariate in covariates:
    sim_params = sim_lstm_test.simulation_info[covariate].values[:n_samples]
    sim_params = np.repeat(sim_params, 20)
    for metric, metric_name in zip(metrics, metric_names):
        plt.figure()
        for i, model_name in enumerate(model_names):
            plt.scatter(sim_params, metric[i], label=model_name, s=.7)
        plt.title(f'{covariate}, {metric_name}')
        plt.legend()


In [ ]:
plt.figure()
var = nmses
plt.plot([0,np.nanmax(var)], [0,np.nanmax(var)], 'orange')
plt.scatter(np.array(var)[1], np.array(var)[2], s=0.5, color='darkblue')
plt.title('nMSE')
plt.xlabel('LSTM')
plt.ylabel('eLORETA')

In [ ]:
idc = [2,3]
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.scatter(nmses[idc[0]], nmses[idc[1]], s=0.5)
ax.plot([0, 1], [0, 1], linewidth=2, color='black')
# ax.set_xlim([-np.percentile(nmses[0], 5), np.percentile(nmses[0], 99)])
# ax.set_ylim([-np.percentile(nmses[0], 5), np.percentile(nmses[0], 99)])
ax.set_xlabel(model_names[idc[0]])
ax.set_ylabel(model_names[idc[1]])

In [ ]:
from scipy.stats import ttest_rel, wilcoxon
print('AUC_far:', ttest_rel(aucs_far[0], aucs_far[1]))
print('AUC_close:', ttest_rel(aucs_close[0], aucs_close[1]))
print('MLE:', wilcoxon(mean_localization_errors[0], mean_localization_errors[1]))
print('nMSE:', ttest_rel(nmses[0], nmses[1]))
print('MSE:', ttest_rel(mses[0], mses[1]))